In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp

X, y = load_keypoints_data_as_kp(
    Path("../download_data/keypoints/validation"),
)
X.shape, y.shape

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
def vis_keypoints(image, keypoints, color=(0, 255, 0)):
    image = image.copy().astype('uint8')
    diameter = int(np.mean([image.shape[0], image.shape[1]])/50)
    for kp in keypoints:
        x, y, = kp[0], kp[1]
        print(x,y)
        if  2 < x < image.shape[0]-2 and 2 < y <  image.shape[1]-2 :
            x_int = int(x)
            y_int = int(y)
            cv2.circle(image, (x_int, y_int), diameter, color, -1)
        else:
            print(f"kp {x},{y} ignored")

    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(image)

vis_keypoints(X[0], y[0])

In [ ]:
image = X[0]
keypoints = y[0]
import albumentations as A
import random
random.seed(7)
transform = A.Compose([
        A.RandomSizedCrop(min_max_height=(128, 224), height=224, width=224, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.HueSaturationValue(p=0.5),
            A.RGBShift(p=0.7)
        ], p=1),
        A.RandomBrightnessContrast(p=0.5)
    ],
    keypoint_params=A.KeypointParams(format='xyas'),
)
transformed = transform(image=image, keypoints=keypoints)
vis_keypoints(transformed['image'], transformed['keypoints'])

In [ ]:
transformed['image'].dtype

In [ ]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((X, y))

from functools import partial

In [ ]:
image, kp = next(iter(dataset))
image = image.numpy()
kp = kp.numpy()
vis_keypoints(image, kp)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
print(y.shape)

In [ ]:
from watch_recognition.augmentations import process_kp_data, set_shapes

ds_alb = dataset.map(process_kp_data, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)


ds_alb = ds_alb.map(
    partial(set_shapes, img_shape=(224, 224, 3), target_shape=tuple(y.shape)[1:]),
    num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
ds_alb

In [ ]:
image, kp = next(iter(ds_alb))
image = image.numpy()
kp = kp.numpy()
vis_keypoints(image, kp)